In [1]:
import pandas as pd
import mysql.connector
pd.options.display.max_columns=None

# Cagamos el fichero, lo analizamos y lo preparamos.

In [11]:
villager_rank = pd.read_csv('acnh_villager_data.csv')
villager_rank.head()

,tier,rank,name
0,1,1,Raymond
1,1,2,Marshal
2,1,3,Shino
3,1,4,Sherb
4,1,5,Sasha


In [12]:
villager = pd.read_csv('ACcatalaog/villagers.csv')
villager.head()

,Name,Species,Gender,Personality,Hobby,Birthday,Catchphrase,Favorite Song,Style 1,Style 2,Color 1,Color 2,Wallpaper,Flooring,Furniture List,Filename,Unique Entry ID
0,Admiral,Bird,Male,Cranky,Nature,27-Jan,aye aye,Steep Hill,Cool,Cool,Black,Blue,dirt-clod wall,tatami,717;1849;7047;2736;787;5970;3449;3622;3802;410...,brd06,B3RyfNEqwGmcccRC3
1,Agent S,Squirrel,Female,Peppy,Fitness,2-Jul,sidekick,Go K.K. Rider,Active,Simple,Blue,Black,concrete wall,colorful tile flooring,7845;7150;3468;4080;290;3971;3449;1708;4756;25...,squ05,SGMdki6dzpDZyXAw5
2,Agnes,Pig,Female,Big Sister,Play,21-Apr,snuffle,K.K. House,Simple,Elegant,Pink,White,gray molded-panel wall,arabesque flooring,4129;7236;7235;7802;896;3428;4027;7325;3958;71...,pig17,jzWCiDPm9MqtCfecP
3,Al,Gorilla,Male,Lazy,Fitness,18-Oct,ayyyeee,Go K.K. Rider,Active,Active,Red,White,concrete wall,green rubber flooring,1452;4078;4013;833;4116;3697;7845;3307;3946;39...,gor08,LBifxETQJGEaLhBjC
4,Alfonso,Alligator,Male,Lazy,Play,9-Jun,it'sa me,Forest Life,Simple,Simple,Red,Blue,yellow playroom wall,green honeycomb tile,4763;3205;3701;1557;3623;85;3208;3584;4761;121...,crd00,REpd8KxB8p9aGBRSE


In [13]:
villager.columns

Index(['Name', 'Species', 'Gender', 'Personality', 'Hobby', 'Birthday',
       'Catchphrase', 'Favorite Song', 'Style 1', 'Style 2', 'Color 1',
       'Color 2', 'Wallpaper', 'Flooring', 'Furniture List', 'Filename',
       'Unique Entry ID'],
      dtype='object')

In [14]:
columnas=['Name', 'Species', 'Gender', 'Personality', 'Hobby', 'Birthday']
df_sql = villager[columnas]

In [15]:
df_sql

,Name,Species,Gender,Personality,Hobby,Birthday
0,Admiral,Bird,Male,Cranky,Nature,27-Jan
1,Agent S,Squirrel,Female,Peppy,Fitness,2-Jul
2,Agnes,Pig,Female,Big Sister,Play,21-Apr
3,Al,Gorilla,Male,Lazy,Fitness,18-Oct
4,Alfonso,Alligator,Male,Lazy,Play,9-Jun
...,...,...,...,...,...,...
386,Winnie,Horse,Female,Peppy,Fashion,31-Jan
387,Wolfgang,Wolf,Male,Cranky,Education,25-Nov
388,Yuka,Koala,Female,Snooty,Fashion,20-Jul
389,Zell,Deer,Male,Smug,Music,7-Jun


In [16]:
def cambio_columnas(dataframe):
    """
   Esta función cambiar el nombre de las columnas a minúscula.
  Args: 
       dataframe(dataframe): el dataframe el cual vamos a cambiar sus columnas 
   Returns: aunque no tiene return, esta función modifica el mismo dataframe con todas las columnas en minúsculas.
   """
    columnas_nuevas = {col :  col.lower() for col in dataframe.columns}
    dataframe.rename(columns = columnas_nuevas, inplace = True)

In [17]:
cambio_columnas(df_sql)

/var/folders/cf/7kf7s8j17wn53znl0ywm912w0000gp/T/ipykernel_3807/1161268461.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.rename(columns = columnas_nuevas, inplace = True)


In [18]:
df_sql

,name,species,gender,personality,hobby,birthday
0,Admiral,Bird,Male,Cranky,Nature,27-Jan
1,Agent S,Squirrel,Female,Peppy,Fitness,2-Jul
2,Agnes,Pig,Female,Big Sister,Play,21-Apr
3,Al,Gorilla,Male,Lazy,Fitness,18-Oct
4,Alfonso,Alligator,Male,Lazy,Play,9-Jun
...,...,...,...,...,...,...
386,Winnie,Horse,Female,Peppy,Fashion,31-Jan
387,Wolfgang,Wolf,Male,Cranky,Education,25-Nov
388,Yuka,Koala,Female,Snooty,Fashion,20-Jul
389,Zell,Deer,Male,Smug,Music,7-Jun


In [19]:
def minus(elemento):
    return elemento.lower()

In [20]:
df_sql['name_index'] = df_sql['name'].apply(minus)

/var/folders/cf/7kf7s8j17wn53znl0ywm912w0000gp/T/ipykernel_3807/1044701161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sql['name_index'] = df_sql['name'].apply(minus)


In [21]:
df_sql = df_sql.set_index('name_index')

# Creamos la clase con todas las funciones necesarias.

In [22]:
class BBDD:
    
    def __init__(self,base_datos='pueblo',tabla = 'vecinos',valor_user='root', valor_password='AlumnaAdalab',valor_host='127.0.0.1'):

        self.base_datos = base_datos.lower()
        self.tabla = tabla.lower()
        self.valor_user = valor_user
        self.valor_password = valor_password
        self.valor_host = valor_host
    
    def crear_base_datos(self):

        eleccion = input('¿Cómo quieres que se llame tu base de datos?')
        self.base_datos = eleccion.lower()

        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()

        try:

            mycursor.execute(f"CREATE DATABASE {self.base_datos}")
            mycursor.execute(f"""CREATE TABLE IF NOT EXISTS {self.tabla}(`id_vecino` INT NOT NULL AUTO_INCREMENT,
                             `nombre` VARCHAR(45) NOT NULL,`especie`VARCHAR(45) NOT NULL,`genero`VARCHAR(45) NOT NULL,
                             `personalidad`VARCHAR(45) NOT NULL,`hobby`VARCHAR(45) NOT NULL,`cumpleaños`VARCHAR(45) NOT NULL,PRIMARY KEY (`id_vecino`))""")
            print(f'Se ha creado la base de datos {self.base_datos}')
            print(f'Estas operando con la base de datos {self.base_datos}')
            print(f'Se ha creado y estás operando con la tabla {self.tabla}')

        except mysql.connector.Error as err:

            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

            opcion = input('¿Quieres crear otra base de datos?(si/no)')

            if opcion.lower() == 'si':
                self.crear_base_datos()

    def cambiar_base_datos(self):


        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        opcion = input(f'Estas operando actualmente con la base de datos {self.base_datos},¿Quieres continuar?(si/no)')

        if opcion.lower() == 'si':
            
            print(f'Las bases de datos disponibles son {bases_de_datos}')

            eleccion = input('Elige la base de datos con la quieres operar')

            if eleccion.lower() in bases_de_datos:

                self.base_datos = eleccion.lower()
                print(f'Estas operando con la base de datos {eleccion}')

            else:

                eleccion_2 = input('Esa eleccion no es correcta, ¿Quieres probar de nuevo? (si/no)')

                if eleccion_2.lower() == 'si':

                    self.cambiar_base_datos()

                else:

                    print(f'La base de datos con la que estás operando es {self.base_datos}')
        else:
            print(f'La base de datos con la que estás operando es {self.base_datos}')

    def crear_tabla(self):

        atencion = input(f'ATENCION: vas a operar con la base de datos {self.base_datos},¿Quieres continuar?(si/no)')
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            if atencion.lower() =='si':

                eleccion = input(f'Especifica el nombre de la tabla que quieres crear en la base de datos {self.base_datos}')
                self.tabla = eleccion.lower()
                cnx = mysql.connector.connect(database=self.base_datos,user=self.valor_user, password = self.valor_password,host = self.valor_host)
                mycursor = cnx.cursor()
                mycursor.execute(f"""CREATE TABLE IF NOT EXISTS {self.tabla}(`id_vecino` INT NOT NULL AUTO_INCREMENT,
                                `nombre` VARCHAR(45) NOT NULL,`especie`VARCHAR(45) NOT NULL,`genero`VARCHAR(45) NOT NULL,
                                `personalidad`VARCHAR(45) NOT NULL,`hobby`VARCHAR(45) NOT NULL,`cumpleaños`VARCHAR(45) NOT NULL,PRIMARY KEY (`id_vecino`))""")

            elif atencion.lower() == 'no':

                eleccion_2 = input('¿Quieres cambiar de base de datos (si/no)')

                if eleccion_2.lower() == 'si':
                    
                    self.cambiar_base_datos()
        else:
            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()


    def cambiar_tabla(self):

        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            cnx = mysql.connector.connect(database=self.base_datos,user=self.valor_user, password = self.valor_password,host = self.valor_host)

            mycursor = cnx.cursor()

            mycursor.execute("SHOW TABLES")

            conocer_tablas = [tablas[0] for tablas in mycursor]
            print(f'Las tablas disponibles son {conocer_tablas}')

            eleccion = input('Elige la tabla con la quieres operar')

            if eleccion.lower() in conocer_tablas:

                self.tabla = eleccion.lower()
                print(f'Estas operando con la base de datos {eleccion.lower()}')

            else:

                eleccion_2 = input('Esa eleccion no es correcta, ¿Quieres probar de nuevo? (si/no)')

                if eleccion_2.lower() == 'si':

                    self.cambiar_tabla()

                else:

                    print(f'La base de datos con la que estás operando es {self.tabla}')
        else:

            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()

    def insertar_vecino(self,df=df_sql):

        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            valores_unicos_especies= list(df['species'].unique())
            valores_unicos_genero= list(df['gender'].unique())
            
            eleccion_especie = input(f'Elige una especie entre estas opciones {valores_unicos_especies}')
            eleccion_genero = input(f'Elige una especie entre estas opciones {valores_unicos_genero}')

            if eleccion_especie.lower().capitalize() in valores_unicos_especies and eleccion_genero.lower().capitalize() in valores_unicos_genero:
                dataframe = df[(df['species']==eleccion_especie.lower().capitalize() ) & (df['gender']==eleccion_genero.lower().capitalize())]
                vecinos = list(dataframe['name'].unique())

                eleccion = input(f'Elige qué vecino quieres añadir {vecinos}')

                if eleccion.lower().capitalize() in vecinos:
                    lista = []
                    for i in df.loc[eleccion.lower(),:].values:
                        lista.append(i)
                    valores = tuple(lista)

                    codigo_sql=f'INSERT INTO {self.tabla} (nombre,especie,genero,personalidad,hobby,cumpleaños)VALUES (%s,%s,%s,%s,%s,%s)'

                    cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                    mycursor = cnx.cursor()
                            
                    try: 
                        mycursor.execute(codigo_sql, valores)
                        cnx.commit()
                        print('Se ha añadido correctamente al vecino en la base de datos.')
                    except mysql.connector.Error as err:
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)
                else:
                    eleccion_3 = input(f'No existe ningún vecino con esas caracteristicas, ¿Quieres probar de nuevo? (si/no)')
                    if eleccion_3.lower() == 'si':
                        self.insertar_vecino()
            else:
                eleccion_3 = input(f'No existe ningún vecino con esas caracteristicas, ¿Quieres probar de nuevo? (si/no)')
                if eleccion_3.lower() == 'si':
                    self.insertar_vecino()
        else:
            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()
    
    def eliminar_vecino(self):

        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:

            cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
            mycursor = cnx.cursor()
            try: 
                mycursor.execute(f'SELECT * FROM {self.tabla}')
            except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)

            nombre_vecino = [elemento[1] for elemento in mycursor]
            eleccion_vecino = input(f'Entre los vecinos disponibles: {nombre_vecino}, ¿Cuál quieres borrar?').lower().capitalize()

            if eleccion_vecino in nombre_vecino:

                codigo_sql=f"DELETE FROM {self.tabla} WHERE nombre = '{eleccion_vecino}'" 
                cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                            
                try: 
                    mycursor.execute(codigo_sql)
                    cnx.commit()
                    print('Se ha eliminado correctamente al vecino en la base de datos.')
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
            else:
                eleccion_3 = input(f'No existe ningún vecino con esas caracteristicas, ¿Quieres probar de nuevo? (si/no)')
                if eleccion_3.lower() == 'si':
                    self.eliminar_vecino()
        else:
            eleccion_3 = input(f'No existe la base de datos {self.base_datos}, ¿Quieres crearla? (si/no)')
            if eleccion_3.lower() == 'si':
                self.crear_base_datos()

    def consultar_vecino(self):
        cnx = mysql.connector.connect(user=self.valor_user, password = self.valor_password,host = self.valor_host)

        mycursor = cnx.cursor()
        mycursor.execute("SHOW DATABASES")

        bases_de_datos = [database[0] for database in mycursor]

        if self.base_datos in bases_de_datos:
            
            atencion = input(f'ATENCION: vas a operar con la base de datos {self.base_datos},¿Quieres continuar?(si/no)')
            
            if atencion.lower() == 'si':

                cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                mycursor = cnx.cursor()
                try: 
                    mycursor.execute(f'SELECT * FROM {self.tabla}')
                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)

                lista_opciones = ['especie','genero','personalidad','hobby','cumpleaños']

                eleccion = input('Entre estos datos: especie,género,personalidad, hobby y cumpleaños, ¿Qué quieres saber?').lower()
                nombre_vecino = [elemento[1] for elemento in mycursor]
                eleccion_vecino = input(f'Entre los vecinos disponibles: {nombre_vecino}, ¿Cuál quieres elegir?').lower().capitalize()

                if eleccion == 'todos' and eleccion_vecino == 'Todos':
                    cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                    mycursor = cnx.cursor()
                    try: 
                        mycursor.execute(f'SELECT * FROM {self.tabla}')
                    except mysql.connector.Error as err:
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)
                    resultado = [elemento[1:] for elemento in mycursor]
                    print('El nombre,género,personalidad,hobby y cumpleaños de todos los vecinos de la base de datos es:')
                    print(resultado)
                
                elif eleccion in lista_opciones and eleccion_vecino == 'Todos':
                    cnx = mysql.connector.connect(user = self.valor_user, password = self.valor_password, host = self.valor_host, database = self.base_datos)
                    mycursor = cnx.cursor()
                    try: 
                        mycursor.execute(f'SELECT * FROM {self.tabla}')
                    except mysql.connector.Error as err:
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)
                    resultado = [elemento[(lista_opciones.index(eleccion))+2] for elemento in mycursor]
                    print(f'La propiedad {eleccion} de todos los vecinos de la base de datos es:')
                    print(resultado)

                elif eleccion == 'todos' and eleccion_vecino in nombre_vecino:

                    try: 
                        mycursor.execute(f"SELECT * FROM {self.tabla} WHERE nombre = '{eleccion_vecino}'")
                    except mysql.connector.Error as err:
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)

                    resultado = [elemento[2:] for elemento in mycursor]
                    print(f'El nombre,género,personalidad,hobby y cumpleaños del vecino {eleccion_vecino} de la base de datos es:')
                    print(resultado)
                elif eleccion in lista_opciones and eleccion_vecino in nombre_vecino:
                    try: 
                        mycursor.execute(f"SELECT * FROM {self.tabla} WHERE nombre = '{eleccion_vecino}'")
                    except mysql.connector.Error as err:
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)

                    resultado = [elemento[(lista_opciones.index(eleccion))+2] for elemento in mycursor]
                    print(f'La propiedad {eleccion} del vecino {eleccion_vecino} de la base de datos es:')
                    print(resultado)
            elif atencion.lower() == 'no':

                eleccion_2 = input('¿Quieres cambiar de base de datos (si/no)')

                if eleccion_2.lower() == 'si':
                    
                    self.cambiar_base_datos()

            

# Probamos las funciones

In [94]:
prueba = BBDD()

In [95]:
prueba.consultar_vecino()

La propiedad genero de todos los vecinos de la base de datos es:
['Bunnie', 'Piper', 'Rolf', 'Boris']


In [45]:
prueba.crear_base_datos()

Se ha creado la base de datos pueblo
Estas operando con la base de datos pueblo
Se ha creado y estás operando con la tabla vecinos


In [30]:
prueba.cambiar_base_datos()

La base de datos con la que estás operando es pueblo


In [ ]:
prueba.crear_tabla()

In [31]:
prueba.cambiar_tabla()

Las tablas disponibles son ['vecinos']
Estas operando con la base de datos vecinos


In [55]:
prueba.insertar_vecino()

Se ha añadido correctamente al vecino en la base de datos.


In [42]:
prueba.eliminar_vecino()

DELETE FROM vecinos WHERE nombre = 'Joey'
Se ha eliminado correctamente al vecino en la base de datos.
